# Review-Based E-Commerce Recommendation Rating Prediction
Addressing a women's clothing e-commerce dataset involves NLP to predict product recommendation ratings. Each record contains a review title, description, rating (1-5), and more.

The focus is binary classification: label 1 for ratings > 3 (recommended) and label 0 for ratings ≤ 3, using review text for prediction.

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, classification_report

### Loading and Viewing Data
[Kaggle Dataset](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews)

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dipanjanS/feature_engineering_session_dhs18/master/ecommerce_product_ratings_prediction/Womens%20Clothing%20E-Commerce%20Reviews.csv', keep_default_na=False)
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


### Data Preprocessing
* Combine review text attributes (title, text description) into a single feature.

* Transform the 5-star rating system into binary recommendations: 1 for ratings > 3 and 0 for ratings ≤ 3.





In [3]:
df['Review'] = (df['Title'].map(str) +' '+ df['Review Text']).apply(lambda row: row.strip())
df['Rating'] = [1 if rating > 3 else 0 for rating in df['Rating']]
df = df[['Review', 'Rating']]
df.head()

,Review,Rating
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,Some major design flaws I had such high hopes ...,0
3,"My favorite buy! I love, love, love this jumps...",1
4,Flattering shirt This shirt is very flattering...,1


### Filtering Records: Eliminating Entries Without Review Text

In [4]:
df = df[df['Review'] != '']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22642 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  22642 non-null  object
 1   Rating  22642 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 530.7+ KB


In [6]:
df['Rating'].value_counts()
# The dataset exhibits some data imbalance concerning product ratings

1    17449
0     5193
Name: Rating, dtype: int64

### Train and Test datasets

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['Review']], df['Rating'], random_state=42)
X_train.shape, X_test.shape

((16981, 1), (5661, 1))

In [8]:
from collections import Counter
Counter(y_train), Counter(y_test)

(Counter({1: 13059, 0: 3922}), Counter({1: 4390, 0: 1271}))

### Part 1: Fundamental NLP Count-Based Features
Basic text-based features can enhance text classification models, including:

* Word Count: total words in the documents
* Character Count: total characters in the documents
* Average Word Density: average word length
* Punctuation Count: total punctuation marks
* Upper Case Count: total uppercase words
* Title Word Count: total proper case (title) words

In [9]:
import string

X_train['char_count'] = X_train['Review'].apply(len)
X_train['word_count'] = X_train['Review'].apply(lambda x: len(x.split()))
X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
X_train['punctuation_count'] = X_train['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
X_train['title_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count'] = X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))


X_test['char_count'] = X_test['Review'].apply(len)
X_test['word_count'] = X_test['Review'].apply(lambda x: len(x.split()))
X_test['word_density'] = X_test['char_count'] / (X_test['word_count']+1)
X_test['punctuation_count'] = X_test['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
X_test['title_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count'] = X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [10]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
12896,Soooo soft! This is a delightfully soft and fl...,268,52,5.056604,8,2,0
13183,"Had my eye on this, but dind't get I finally v...",399,84,4.694118,20,2,1
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0
13366,"Boxy. large. Boxy, unflattering, and large.\n\...",295,51,5.673077,22,2,0


### Training a Logistic Regression Model

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1, random_state=42, solver='liblinear')

### Metrics for Model Assessment
* Precision: Correct predictions in a class
* Recall: Correctly predicted class instances
* F1-Score: Harmonic mean of precision and recall



In [12]:
lr.fit(X_train.drop(['Review'], axis=1), y_train)
predictions = lr.predict(X_test.drop(['Review'], axis=1))

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1271
           1       0.78      1.00      0.87      4390

    accuracy                           0.78      5661
   macro avg       0.39      0.50      0.44      5661
weighted avg       0.60      0.78      0.68      5661



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1
0,0,1271
1,0,4390


Our model didn't predict any products with unfavorable (no recommendation) ratings, Class 0, resembling uniform positive predictions.


### Leveraging Text Sentiment
Text reviews are rich in subjective content, making sentiment extraction a valuable feature. High-rated, recommended products (label 1) often correspond to positive sentiment, while non-recommended products (label 0) lean towards negative sentiment. TextBlob offers convenient sentiment analysis using a polarity score (-1.0 to 1.0) and subjectivity score (0.0 to 1.0).

In [13]:
import textblob

textblob.TextBlob('This is an AMAZING pair of Jeans!').sentiment

Sentiment(polarity=0.7500000000000001, subjectivity=0.9)

In [14]:
textblob.TextBlob('I really hated this UGLY T-shirt!!').sentiment

Sentiment(polarity=-0.95, subjectivity=0.85)

This approach seems promising for deriving differentiating features between favorable and unfavorable products.

### Part 2: Features from Sentiment Analysis
This is unsupervised sentiment analysis using a lexicon. We lack pre-labeled data; the lexicon guides sentiment determination.

In [15]:
x_train_snt_obj = X_train['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
X_train['Subjectivity'] = [obj.subjectivity for obj in x_train_snt_obj.values]

x_test_snt_obj = X_test['Review'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_test['Polarity'] = [obj.polarity for obj in x_test_snt_obj.values]
X_test['Subjectivity'] = [obj.subjectivity for obj in x_test_snt_obj.values]

In [16]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
12896,Soooo soft! This is a delightfully soft and fl...,268,52,5.056604,8,2,0,0.170455,0.490909
13183,"Had my eye on this, but dind't get I finally v...",399,84,4.694118,20,2,1,0.101944,0.719537
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2,0.186538,0.458761
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0,0.625000,0.825000
13366,"Boxy. large. Boxy, unflattering, and large.\n\...",295,51,5.673077,22,2,0,0.329613,0.510268


### Model Training and Evaluation

In [17]:
lr.fit(X_train.drop(['Review'], axis=1), y_train, )
predictions = lr.predict(X_test.drop(['Review'], axis=1))

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.69      0.27      0.38      1271
           1       0.82      0.97      0.89      4390

    accuracy                           0.81      5661
   macro avg       0.75      0.62      0.64      5661
weighted avg       0.79      0.81      0.77      5661



,0,1
0,338,933
1,153,4237


Now we predict 27% of negative-rated products, with 69% precision. Bad reviews show 40% F1-Score and good reviews, 89%. Overall, F1-Score reaches 77%, commendable. Yet, room for enhancement exists, as bad review recall remains low.

### Text Pre-processing and Wrangling
Extracting targeted features through NLP's common Bag of Words model necessitates text data cleaning. We'll create a basic text pre-processor for feature strategy review, concentrating on:
* Text Lowercasing
* Contraction Removal
* Clearing extraneous characters, digits, symbols
* Stemming
* Stopword elimination

In [18]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
import contractions

contractions.fix('I didn\'t like this t-shirt')

'I did not like this t-shirt'

In [20]:
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

def simple_text_preprocessor(document):
    # lower case
    document = str(document).lower()

    # expand contractions
    document = contractions.fix(document)

    # remove unnecessary characters
    document = re.sub(r'[^a-zA-Z]',r' ', document)
    document = re.sub(r'nbsp', r'', document)
    document = re.sub(' +', ' ', document)

    # simple porter stemming
    document = ' '.join([ps.stem(word) for word in document.split()])

    # stopwords removal
    document = ' '.join([word for word in document.split() if word not in stop_words])

    return document

stp = np.vectorize(simple_text_preprocessor)

In [21]:
X_train['Clean Review'] = stp(X_train['Review'].values)
X_test['Clean Review'] = stp(X_test['Review'].values)

X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,Clean Review
12896,Soooo soft! This is a delightfully soft and fl...,268,52,5.056604,8,2,0,0.170455,0.490909,soooo soft thi delight soft fluffi sweater mig...
13183,"Had my eye on this, but dind't get I finally v...",399,84,4.694118,20,2,1,0.101944,0.719537,eye thi but dind get final visit store petit t...
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2,0.186538,0.458761,want like thi want like thi top badli badli fa...
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0,0.625000,0.825000,beauti blous bought thi daughter law birthday ...
13366,"Boxy. large. Boxy, unflattering, and large.\n\...",295,51,5.673077,22,2,0,0.329613,0.510268,boxi larg boxi unflatt larg curvi pound thi to...


### Deriving Structured Features from Previous Experiments

In [22]:
X_train_metadata = X_train.drop(['Review', 'Clean Review'], axis=1).reset_index(drop=True)
X_test_metadata = X_test.drop(['Review', 'Clean Review'], axis=1).reset_index(drop=True)

X_train_metadata.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
0,268,52,5.056604,8,2,0,0.170455,0.490909
1,399,84,4.694118,20,2,1,0.101944,0.719537
2,525,104,5.000000,19,2,2,0.186538,0.458761
3,203,35,5.638889,10,2,0,0.625000,0.825000
4,295,51,5.673077,22,2,0,0.329613,0.510268


### Part 3: Integrating Bag of Words Features - Unigram Approach

A simple vector space model for unstructured text represents data as numeric vectors, with each dimension as a distinct feature. The bag of words model signifies text documents as numeric vectors, where dimensions correspond to corpus words, indicating frequency, occurrence (1/0), or weighted values. It's named as such since documents are treated as 'bags' of words, neglecting order and grammar.

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean Review']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names_out())

X_testcv = cv.transform(X_test['Clean Review']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names_out())
X_traincv.head()

,aa,aaaaandidon,aaaaannnnnnd,aaaah,aaaahmaz,aaah,ab,abbey,abbi,abck,...,zing,zip,zipper,zipperi,zippi,zone,zooland,zoom,zowi,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
X_test_comb = pd.concat([X_test_metadata, X_testcv], axis=1)

X_train_comb.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,aa,aaaaandidon,...,zing,zip,zipper,zipperi,zippi,zone,zooland,zoom,zowi,zuma
0,268,52,5.056604,8,2,0,0.170455,0.490909,0,0,...,0,0,0,0,0,0,0,0,0,0
1,399,84,4.694118,20,2,1,0.101944,0.719537,0,0,...,0,0,0,0,0,0,0,0,0,0
2,525,104,5.000000,19,2,2,0.186538,0.458761,0,0,...,0,0,0,0,0,0,0,0,0,0
3,203,35,5.638889,10,2,0,0.625000,0.825000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,295,51,5.673077,22,2,0,0.329613,0.510268,0,0,...,0,0,0,0,0,0,0,0,0,0


### Model Training and Evaluation

In [27]:
lr.fit(X_train_comb, y_train)
predictions = lr.predict(X_test_comb)

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.76      0.70      0.73      1271
           1       0.92      0.93      0.93      4390

    accuracy                           0.88      5661
   macro avg       0.84      0.82      0.83      5661
weighted avg       0.88      0.88      0.88      5661



,0,1
0,896,375
1,286,4104


We achieved 70% prediction for negative-rated products with 76% precision. Bad reviews show 73% F1-Score, while good reviews reach 92%. Our overall F1-Score is a solid 88%.